In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
all_df = pd.read_json('/content/drive/MyDrive/LinkData/dtp_uz/Database/dtp_uz.json')

In [ ]:

def new_dict_(k, v, parent_key=''):
    """
    Function to flatten a dictionary with nested structures.
    """
    items = []
    new_key = f"{parent_key}_{k}" if parent_key else k

    if isinstance(v, dict):
        for sub_key, sub_value in v.items():
            items.extend(new_dict_(sub_key, sub_value, new_key).items())
    elif isinstance(v, list):
        for num, elem in enumerate(v, start=1):
            if isinstance(elem, dict):
                items.extend(new_dict_(num, elem, new_key).items())
            else:
                items.append((f"{new_key}_{num}", elem))
    else:
        items.append((new_key, v))

    return dict(items)

# Assuming all_df['results'][0] is a dictionary

all_data = []
for index_ in all_df.index:
    new_dict = {}
    for key, value in all_df['results'][index_].items():
        new_dict.update(new_dict_(key, value))

    # Now new_dict contains the flattened structure

    new_dict.update({'num_participants' : len(all_df['results'][0]['participants'])})
    new_dict.update({'num_vehicles' : len(all_df['results'][0]['vehicles'])})
    all_data.append(new_dict)
    # len(new_dict)
len(all_data)

In [ ]:
df_need = pd.DataFrame(all_data)
df_need.head()

In [ ]:
# To SQL
import sqlite3
conn = sqlite3.connect("/content/drive/MyDrive/LinkData/dtp_uz/Database/dtp_uz.db")
df_need.to_sql("dtp_uz", conn, if_exists='append', index=False)

In [2]:
# Read SQL
import sqlite3
import pandas as pd

conn = sqlite3.connect("/content/drive/MyDrive/LinkData/dtp_uz/Database/dtp_uz.db")
df_need_sql = pd.read_sql("SELECT * FROM dtp_uz", conn)

In [ ]:
print(df_need_sql.shape)
df_need_sql.head()

In [4]:
df_need_sql_ = df_need_sql.copy()

In [39]:
df_need_sql = df_need_sql_.copy()

In [5]:
# Drop columns with NaNs 100% of the time

df_need_sql = df_need_sql.applymap(lambda x: x if x != '' else None)

cols_ = df_need_sql.columns[df_need_sql.isna().all()]

df_need_sql.dropna(axis=1, how='all', inplace=True)

filter_accident_number = df_need_sql['accident_number'].isna()
df_need_sql.drop(df_need_sql[filter_accident_number].index, inplace=True)


df_need_sql['year_'] = df_need_sql['date_accident'].apply(lambda x: int(x[:4]))

filter_year = df_need_sql['year_'] < 2010
df_need_sql.drop(df_need_sql[filter_year].index, inplace=True)
# df_need_sql['year_'].value_counts()


In [ ]:
col_need = df_need_sql.columns
col_need = col_need[col_need.str.contains('violation') & col_need.str.contains('id')]

# Col string
df_need_sql[col_need] = df_need_sql[col_need].applymap(str)

# Set
df_need_sql['violation_sets'] = df_need_sql.loc[:, col_need].apply(lambda x: set(x), axis=1)

# Discard None
for i in df_need_sql.index:
    df_need_sql['violation_sets'][i].discard('nan')

# Function to convert list to string and remove NaNs
def convert_list_to_string(lst):
    # Filter out NaN values and convert each item to string
    return '/'.join([str(item) for item in lst if pd.notna(item)])

# df_need_sql['violation_sets'] = df_need_sql['violation_sets'].apply(list)
df_need_sql['len_violation_sets'] = df_need_sql['violation_sets'].apply(len)

# Apply the function to the violation_sets column
df_need_sql['violation_sets'] = df_need_sql['violation_sets'].apply(convert_list_to_string)



print(df_need_sql['len_violation_sets'].unique())
print()
df_need_sql.head()

In [ ]:
col_need = df_need_sql.columns
col_need = col_need[col_need.str.contains('participants') & col_need.str.contains('gender')]

# # Col string
df_need_sql[col_need] = df_need_sql[col_need].applymap(str)

# Set
df_need_sql['gender_list'] = df_need_sql.loc[:, col_need].apply(lambda x: list(x), axis=1)

# Drop None or NaN values from lists in the 'violation_sets' column
df_need_sql['gender_list'] = df_need_sql['gender_list'].apply(lambda x: [item for item in x if item != 'None'])
df_need_sql['len_gender_list'] = df_need_sql['gender_list'].apply(len)

# Function to count occurrences of 'Male' and 'Female' in a list
def count_gender_occurrences(lst):
    male_count = lst.count('male')
    female_count = lst.count('female')
    return male_count, female_count

# Apply the count_gender_occurrences function to each list in the 'gender_list' column
df_need_sql[['male_count', 'female_count']] = df_need_sql['gender_list'].apply(lambda x: pd.Series(count_gender_occurrences(x)))

df_need_sql.iloc[:, -5:]

In [ ]:
from datetime import datetime

# Convert the 'datetime_column' to datetime objects
df_need_sql['date_accident'] = pd.to_datetime(df_need_sql['date_accident'], format="%Y-%m-%dT%H:%M:%S")

# Extract hours from the datetime column
df_need_sql['day_of_week'] = df_need_sql['date_accident'].apply(lambda x: x.weekday())
df_need_sql['day_name'] = df_need_sql['date_accident'].apply(lambda x: x.strftime("%A"))
df_need_sql['day_hour'] = df_need_sql['date_accident'].apply(lambda x: x.hour)

df_need_sql['day_name'] = df_need_sql['day_name'].str.replace("Monday", "Dushanba")\
                                               .str.replace("Tuesday", "Seshanba")\
                                               .str.replace("Wednesday", "Chorshanba")\
                                               .str.replace("Thursday", "Payshanba")\
                                               .str.replace("Friday", "Juma")\
                                               .str.replace("Saturday", "Shanba")\
                                               .str.replace("Sunday", "Yakshanba")

df_need_sql.iloc[:,-10:]


In [ ]:
df_need_sql.shape

In [ ]:
# To SQL
import pandas as pd
import sqlite3

conn = sqlite3.connect("/content/drive/MyDrive/LinkData/dtp_uz/Database/dtp_uz_27012024.db")
df_need_sql.to_sql("dtp_uz", conn, if_exists='append', index=False)